# 🧠 Análisis de la Readmisión a la UCI en Pacientes con Hemorragia Intracerebral (MIMIC-IV)

## 1. 🔧 Configuración Inicial

In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Path to your service account key file
SERVICE_ACCOUNT_FILE = "mimic-sergi.json"
PROJECT_ID = "ogi-uci-i61"

# Create credentials and client
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE)

client = bigquery.Client(credentials=credentials, project=PROJECT_ID)

# Example query
query = "SELECT 'Hello, World!' AS greeting"
query_job = client.query(query)

# Fetch results
results = query_job.result()

for row in results:
    print(row.greeting)

Hello, World!


## 2. 📦 Extracción de Datos

In [ ]:
# SQL query para extraer datos de pacientes con HIC (Hemorragia Intracerebral) en la UCI
query = """
-- Updated SQL query for ICH patients in ICU with corrected joins to derived lab tables
WITH ich_admissions AS (
  SELECT DISTINCT hadm_id
  FROM `ogi-uci-i61.mimiciv_hosp.diagnoses_icd` i
  JOIN `ogi-uci-i61.mimiciv_hosp.d_icd_diagnoses` d
    ON i.icd_code = d.icd_code AND i.icd_version = d.icd_version
  WHERE LOWER(d.long_title) LIKE '%intracerebral hemorrhage%'
),

chartevents_avg AS (
  SELECT
    c.stay_id,
    AVG(CASE WHEN itemid = 220045 AND valuenum > 0 THEN valuenum END) AS heart_rate_avg,
    AVG(CASE WHEN itemid = 220052 AND valuenum BETWEEN 30 AND 200 THEN valuenum END) AS mbp_avg,
    AVG(CASE WHEN itemid = 220210 AND valuenum BETWEEN 5 AND 60 THEN valuenum END) AS resp_rate_avg,
    AVG(CASE WHEN itemid = 220277 AND valuenum BETWEEN 70 AND 100 THEN valuenum END) AS spo2_avg,
    AVG(CASE WHEN itemid = 223762 AND valuenum BETWEEN 33 AND 43 THEN valuenum END) AS temperature_avg
  FROM `ogi-uci-i61.mimiciv_icu.chartevents` c
  JOIN `ogi-uci-i61.mimiciv_icu.icustays` i ON c.stay_id = i.stay_id
  WHERE
    c.itemid IN (220045, 220052, 220210, 220277, 223762)
    AND c.valuenum IS NOT NULL
    AND c.charttime BETWEEN i.intime AND i.outtime
  GROUP BY c.stay_id
),

cbc_labs AS (
  SELECT subject_id, hadm_id,
    MIN(wbc) AS wbc_min,
    MIN(hemoglobin) AS hgb_min,
    MIN(hematocrit) AS hct_min
  FROM `ogi-uci-i61.mimiciv_derived.complete_blood_count`
  GROUP BY subject_id, hadm_id
),

chem_labs AS (
  SELECT subject_id, hadm_id,
    MIN(sodium) AS sodium_min,
    MIN(creatinine) AS creatinine_min,
    MIN(glucose) AS glucose_min
  FROM `ogi-uci-i61.mimiciv_derived.chemistry`
  GROUP BY subject_id, hadm_id
),

coag_labs AS (
  SELECT subject_id, hadm_id,
    MIN(inr) AS inr_min,
    MIN(pt) AS pt_min
  FROM `ogi-uci-i61.mimiciv_derived.coagulation`
  GROUP BY subject_id, hadm_id
),

readmission_within_30_days AS (
  SELECT
    a1.subject_id,
    a1.stay_id,
    MIN(a2.stay_id) AS readmitted_stay
  FROM `ogi-uci-i61.mimiciv_icu.icustays` a1
  JOIN `ogi-uci-i61.mimiciv_icu.icustays` a2
    ON a1.subject_id = a2.subject_id
    AND a2.intime > a1.outtime
    AND DATETIME_DIFF(a2.intime, a1.outtime, DAY) <= 30
  GROUP BY a1.subject_id, a1.stay_id
)

SELECT
  a.subject_id,
  a.hadm_id,
  a.stay_id,
  p.gender,
  p.anchor_age AS age,
  a.intime,
  a.outtime,
  a.first_careunit,
  TIMESTAMP_DIFF(a.outtime, a.intime, HOUR) AS icu_los_hours,
  TIMESTAMP_DIFF(adm.dischtime, adm.admittime, HOUR) AS hosp_los_hours,
  adm.discharge_location,
  adm.hospital_expire_flag,

  STRING_AGG(CASE WHEN LOWER(d.long_title) LIKE '%intracerebral hemorrhage%' THEN i.icd_code END, '; ') AS ich_icd_codes,
  STRING_AGG(CASE WHEN LOWER(d.long_title) LIKE '%intracerebral hemorrhage%' THEN CONCAT(i.icd_code, ': ', d.long_title) ELSE i.icd_code END, '; ') AS all_diagnoses,

  s.gcs_verbal,
  s.gcs_motor,
  s.gcs_eyes,

  vitals.heart_rate,
  vitals.mbp,
  vitals.resp_rate,
  vitals.temperature,
  vitals.spo2,
  ce.heart_rate_avg,
  ce.mbp_avg,
  ce.resp_rate_avg,
  ce.spo2_avg,
  ce.temperature_avg,

  -- Derived Scores and Labs
  charlson.charlson_comorbidity_index,
  apsi.apsiii,
  code_status.dnr,
  code_status.dni,
  code_status.fullcode,
  code_status.cmo,
  cbc.wbc_min, cbc.hgb_min, cbc.hct_min,
  chem.sodium_min, chem.creatinine_min, chem.glucose_min,
  coag.inr_min, coag.pt_min,

  CASE WHEN hyper.hadm_id IS NOT NULL THEN 1 ELSE 0 END AS has_hypertension,
  CASE WHEN hydro.hadm_id IS NOT NULL THEN 1 ELSE 0 END AS has_hydrocephalus,
  CASE WHEN cad.hadm_id IS NOT NULL THEN 1 ELSE 0 END AS has_cad,
  CASE WHEN anticoag.subject_id IS NOT NULL THEN 1 ELSE 0 END AS has_anticoagulation,
  CASE WHEN neuroproc.hadm_id IS NOT NULL THEN 1 ELSE 0 END AS had_neurosurgery,
  CASE WHEN addmeds.subject_id IS NOT NULL THEN 1 ELSE 0 END AS on_statins_or_antiplatelets,

  readmit.readmitted_stay

FROM `ogi-uci-i61.mimiciv_icu.icustays` a
JOIN `ogi-uci-i61.mimiciv_hosp.patients` p ON a.subject_id = p.subject_id
JOIN `ogi-uci-i61.mimiciv_hosp.admissions` adm ON a.hadm_id = adm.hadm_id
JOIN ich_admissions ich ON a.hadm_id = ich.hadm_id
LEFT JOIN `ogi-uci-i61.mimiciv_hosp.diagnoses_icd` i ON a.hadm_id = i.hadm_id
LEFT JOIN `ogi-uci-i61.mimiciv_hosp.d_icd_diagnoses` d ON i.icd_code = d.icd_code AND i.icd_version = d.icd_version

LEFT JOIN (
  SELECT stay_id, gcs_verbal, gcs_motor, gcs_eyes,
         ROW_NUMBER() OVER (PARTITION BY stay_id ORDER BY charttime) AS rn
  FROM `ogi-uci-i61.mimiciv_derived.gcs`
) s ON a.stay_id = s.stay_id AND s.rn = 1

LEFT JOIN (
  SELECT stay_id, heart_rate, mbp, resp_rate, temperature, spo2,
         ROW_NUMBER() OVER (PARTITION BY stay_id ORDER BY charttime) AS rn
  FROM `ogi-uci-i61.mimiciv_derived.vitalsign`
) vitals ON a.stay_id = vitals.stay_id AND vitals.rn = 1

LEFT JOIN chartevents_avg ce ON a.stay_id = ce.stay_id
LEFT JOIN readmission_within_30_days readmit ON a.subject_id = readmit.subject_id AND a.stay_id = readmit.stay_id

-- Derived Scores and Corrected Lab Joins
LEFT JOIN `ogi-uci-i61.mimiciv_derived.charlson` charlson ON a.hadm_id = charlson.hadm_id
LEFT JOIN `ogi-uci-i61.mimiciv_derived.apsiii` apsi ON a.stay_id = apsi.stay_id
LEFT JOIN `ogi-uci-i61.mimiciv_derived.code_status` code_status ON a.stay_id = code_status.stay_id
LEFT JOIN cbc_labs cbc ON a.subject_id = cbc.subject_id AND a.hadm_id = cbc.hadm_id
LEFT JOIN chem_labs chem ON a.subject_id = chem.subject_id AND a.hadm_id = chem.hadm_id
LEFT JOIN coag_labs coag ON a.subject_id = coag.subject_id AND a.hadm_id = coag.hadm_id

LEFT JOIN (
  SELECT DISTINCT hadm_id
  FROM `ogi-uci-i61.mimiciv_hosp.diagnoses_icd`
  WHERE icd_code LIKE '401%' OR icd_code LIKE 'I10%'
) hyper ON a.hadm_id = hyper.hadm_id

LEFT JOIN (
  SELECT DISTINCT hadm_id
  FROM `ogi-uci-i61.mimiciv_hosp.diagnoses_icd`
  WHERE icd_code LIKE '331.3' OR icd_code LIKE 'G91%'
) hydro ON a.hadm_id = hydro.hadm_id

LEFT JOIN (
  SELECT DISTINCT hadm_id
  FROM `ogi-uci-i61.mimiciv_hosp.diagnoses_icd`
  WHERE icd_code LIKE '414%' OR icd_code LIKE 'I25%'
) cad ON a.hadm_id = cad.hadm_id

LEFT JOIN (
  SELECT DISTINCT subject_id
  FROM `ogi-uci-i61.mimiciv_hosp.prescriptions`
  WHERE REGEXP_CONTAINS(LOWER(drug), r'(warfarin|heparin|apixaban|rivaroxaban|dabigatran)')
    AND drug_type = 'MAIN'
) anticoag ON a.subject_id = anticoag.subject_id

LEFT JOIN (
  SELECT DISTINCT subject_id
  FROM `ogi-uci-i61.mimiciv_hosp.prescriptions`
  WHERE REGEXP_CONTAINS(LOWER(drug), r'(aspirin|clopidogrel|atorvastatin|rosuvastatin)')
    AND drug_type = 'MAIN'
) addmeds ON a.subject_id = addmeds.subject_id

LEFT JOIN (
  SELECT DISTINCT hadm_id
  FROM `ogi-uci-i61.mimiciv_hosp.procedures_icd`
  WHERE icd_code LIKE '01%' OR icd_code LIKE '02%'
) neuroproc ON a.hadm_id = neuroproc.hadm_id

GROUP BY
  a.subject_id, a.hadm_id, a.stay_id, a.intime, a.outtime, a.first_careunit,
  p.gender, p.anchor_age, adm.admittime, adm.dischtime, adm.discharge_location, adm.hospital_expire_flag,
  s.gcs_verbal, s.gcs_motor, s.gcs_eyes,
  vitals.heart_rate, vitals.mbp, vitals.resp_rate, vitals.temperature, vitals.spo2,
  ce.heart_rate_avg, ce.mbp_avg, ce.resp_rate_avg, ce.spo2_avg, ce.temperature_avg,
  charlson.charlson_comorbidity_index,
  apsi.apsiii,
  code_status.dnr, code_status.dni, code_status.fullcode, code_status.cmo,
  cbc.wbc_min, cbc.hgb_min, cbc.hct_min,
  chem.sodium_min, chem.creatinine_min, chem.glucose_min,
  coag.inr_min, coag.pt_min,
  hyper.hadm_id, hydro.hadm_id, cad.hadm_id,
  anticoag.subject_id, addmeds.subject_id, neuroproc.hadm_id,
  readmit.readmitted_stay

"""

# Ejecutar la consulta
df = client.query(query).to_dataframe()

df

## 3. 🧽 Preprocesamiento

In [ ]:
import pandas as pd

# Create target variable (readmitted)
print("\nNulls in readmitted_stay before fillna:", df['readmitted_stay'].isnull().sum())

# Check how many unique stay IDs there are
unique_stay_ids = df['stay_id'].nunique()
print(f"\nNumber of unique stay IDs: {unique_stay_ids}")

# Check for duplicate stay IDs (if any)
duplicate_stay_ids = df['stay_id'].duplicated().sum()
print(f"Number of duplicate stay IDs: {duplicate_stay_ids}")

# Check how many unique hadm IDs there are
unique_hadm_ids = df['hadm_id'].nunique()
print(f"\nNumber of unique hadm IDs: {unique_hadm_ids}")

# Check for duplicate stay IDs (if any)
duplicate_hadm_ids = df['hadm_id'].duplicated().sum()
print(f"Number of duplicate hadm IDs: {duplicate_hadm_ids}")


# Define readmitted (1 if readmitted_stay is not null, 0 otherwise)
df['readmitted'] = df['readmitted_stay'].notnull().astype(int)  # Or (df['readmitted_stay'] > 0).astype(int)
print("Class distribution in readmitted:", df['readmitted'].value_counts())

# Explicitly convert icu_los_hours to numerical type
df['icu_los_hours'] = pd.to_numeric(df['icu_los_hours'])

# Convert to categorical 
df['gcs_verbal'] = df['gcs_verbal'].astype('category')
df['gcs_motor'] = df['gcs_motor'].astype('category')
df['gcs_eyes'] = df['gcs_eyes'].astype('category')
df['gender'] = df['gender'].astype('category')

# Split semicolon-separated codes into lists for all diagnoses
df['all_diagnoses_list'] = df['all_diagnoses'].str.split(';')

# Get all ICD codes and their frequencies
all_codes = df['all_diagnoses_list'].explode().str.strip()
# Extract just the ICD code (before the colon, if present)
all_codes = all_codes.str.split(':').str[0].str.strip()
all_codes = all_codes[all_codes != '']  # Exclude empty strings

# Filter out ICH-related codes
# Assuming ICH codes are those in ich_icd_codes or identified by the SQL condition
ich_codes = df['ich_icd_codes'].str.split(';').explode().str.strip().unique()
ich_codes = [code for code in ich_codes if code]  # Remove empty strings
non_ich_codes = all_codes[~all_codes.isin(ich_codes)]

# Get frequency of non-ICH codes
non_ich_code_counts = non_ich_codes.value_counts()

# Identify the 10 most frequent non-ICH codes
top_10_non_ich_codes = non_ich_code_counts.head(20).index.tolist()
print(f"\nTop 10 most frequent non-ICH ICD-10 codes: {top_10_non_ich_codes}")
print("Frequency of non-ICH codes:")
print(non_ich_code_counts.head(20))

# One-hot encode the top 10 non-ICH codes
for code in top_10_non_ich_codes:
    df[f'non_ich_{code}'] = df['all_diagnoses'].str.contains(code, na=False).astype(int)

# Drop temporary column
df = df.drop(columns=['all_diagnoses_list'], errors='ignore')

# Optionally, if you still want to one-hot encode ICH codes (as in your original code)
df['ich_icd_codes_list'] = df['ich_icd_codes'].str.split(';')
all_ich_codes = df['ich_icd_codes_list'].explode().str.strip()
all_ich_codes = all_ich_codes[all_ich_codes != '']
ich_code_counts = all_ich_codes.value_counts()
top_7_ich_codes = ich_code_counts.head(7).index.tolist()
print(f"\nTop 7 most frequent ICH ICD-10 codes: {top_7_ich_codes}")
print("Frequency of ICH codes:")
print(ich_code_counts)

for code in top_7_ich_codes:
    df[f'ich_{code}'] = df['ich_icd_codes'].str.contains(code, na=False).astype(int)

# Drop temporary ICH column
df = df.drop(columns=['ich_icd_codes_list'], errors='ignore')

df

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Primer hauríem de mirar quins tipus de dades conté cada columna
print(df.dtypes)



subject_id                  Int64
hadm_id                     Int64
stay_id                     Int64
gender                     object
intime             datetime64[us]
outtime            datetime64[us]
icd_code                   object
icd_version                 Int64
long_title                 object
gcs_verbal                float64
gcs_motor                 float64
gcs_eyes                  float64
heart_rate                float64
mbp                       float64
resp_rate                 float64
temperature                object
spo2                      float64
readmitted_stay             Int64
dtype: object


## 4. 📊 Modelado Predictivo

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from decimal import Decimal

# Calculate missing percentages
missing_pct = (df.isnull().sum() / len(df) * 100).round(2)
print("Missing percentages per column:")
print(missing_pct.sort_values(ascending=False))

# Set missing data threshold
missing_threshold = 60  # Exclude columns with >60% missing
columns_to_exclude = missing_pct[missing_pct > missing_threshold].index
print(f"\nColumns with >{missing_threshold}% missing: {list(columns_to_exclude)}")

# Exclude high-missingness columns, but keep readmitted_stay for target creation
features_to_keep = [col for col in df.columns if col not in columns_to_exclude or col == 'readmitted_stay']
df_clean = df.loc[:, features_to_keep]

# Handle Decimal values and fill NaNs
# Convert Decimal to float for numerical columns
numerical_cols = df_clean.select_dtypes(include=['number', 'Int64', 'float64', 'int64']).columns
for col in numerical_cols:
    df_clean.loc[:, col] = df_clean[col].apply(lambda x: float(x) if isinstance(x, Decimal) else x)

# Identify integer columns
int_columns = df_clean.select_dtypes(include=['Int64', 'int64']).columns
# Fill NaNs with rounded medians for integer columns
for col in int_columns:
    median_val = df_clean[col].median()
    if not pd.isna(median_val):
        df_clean.loc[:, col] = df_clean[col].fillna(int(median_val))

# Fill NaNs for other numeric columns, excluding readmitted_stay and readmitted
numeric_cols = df_clean.select_dtypes(include='number').columns.difference(['readmitted_stay', 'readmitted'])
df_clean.loc[:, numeric_cols] = df_clean[numeric_cols].fillna(df_clean[numeric_cols].median())

# Handle readmitted_stay separately
df_clean.loc[:, 'readmitted_stay'] = df_clean['readmitted_stay'].fillna(0)

# Scale features
scaler = StandardScaler()
feature_cols = df_clean.select_dtypes(include='number').columns[
    ~df_clean.select_dtypes(include='number').columns.str.startswith(('non_ich', 'ich'))
].difference(['readmitted_stay', 'readmitted', 'subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime', 'has_hypertension', 'has_hydrocephalus', 'has_cad', 'has_anticoagulation', 'had_neurosurgery'])
features_scaled = scaler.fit_transform(df_clean[feature_cols])
df_clean.loc[:, feature_cols] = features_scaled

# Remove columns that are not needed for the final dataset 
df_clean = df_clean.drop(columns=['intime', 'outtime', 'subject_id', 'hadm_id', 'stay_id', 'readmitted_stay', 'all_diagnoses','ich_icd_codes'], errors='ignore')

df_clean


Nulls in readmitted_stay before fillna: 3222
readmitted_stay summary: count              938.0
mean     35030430.127932
std       2878265.870109
min           30024491.0
25%           32506322.0
50%           34949251.5
75%          37500054.75
max           39979862.0
Name: readmitted_stay, dtype: Float64
Class distribution in readmitted: readmitted
0    3222
1     938
Name: count, dtype: int64


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from pycaret.classification import *

# Prepare data for PyCaret
ordinal_features = {
    'gcs_verbal': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0],
    'gcs_motor':  [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0],
    'gcs_eyes':   [0.0, 1.0, 2.0, 3.0, 4.0]
}
feature_cols = df_clean.columns.difference(['readmitted'])
X = df_clean[feature_cols]
y = df_clean['readmitted']
print("y value counts:", y.value_counts())
print("y unique values:", y.unique())

clf_setup = setup(
    data=df_clean,
    target='readmitted',
    ordinal_features=ordinal_features,
    normalize=False,
    session_id=42,
    fix_imbalance=True,
)

In the initial round of model evaluation, most classifiers achieved high accuracy (around 77%) but extremely low recall and F1 scores. This indicated that the models were predominantly predicting the majority class (i.e., not readmitted), failing to identify actual readmission cases. For example, Ridge Classifier and Logistic Regression had recall values below 2%, and the Dummy Classifier set a high baseline accuracy simply by predicting all negatives. These results highlighted the impact of class imbalance and the need for better strategies to detect minority class cases.

In [15]:
# Paso 4: Comparar Modelos
print("\nComparant models........ esperant........ pot tardar uns minuts...")
best_model = compare_models()


Comparant models........ esperant........ pot tardar uns minuts...


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.7819,0.6383,0.0762,0.6342,0.1352,0.0924,0.1625,0.0610
lightgbm,Light Gradient Boosting Machine,0.7785,0.6018,0.1554,0.5465,0.2391,0.1539,0.1974,0.4460
dummy,Dummy Classifier,0.7744,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0120
dt,Decision Tree Classifier,0.6776,0.5529,0.3258,0.3023,0.3128,0.1031,0.1034,0.0120


## 5. 📈 Visualización de Resultados

## 6. 🚀 Despliegue (Futuro)
Se puede utilizar Hugging Face Spaces o Streamlit Cloud para desplegar un frontend que permita:
- Cargar variables clínicas
- Obtener un score de riesgo
- Visualizar el gráfico ROC o una matriz de confusión